In [1]:
import h5py
import rasterio
from rasterio.mask import mask
from rasterio.transform import from_origin
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd 
import pandas as pd
from dnb_annual import *
from variables import years, composites, region_map, region_names

c:\Users\jakub\anaconda\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\jakub\anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
# this script is used only once to generate the regional images for each year
# country_polygons = gpd.read_file("geoBoundaries-UKR-ADM1.geojson")

# for year in years:
#     dnb = dnb_annual(year, composites, country_polygons)
#     dnb.load_all_data()
#     dnb.save_rasters()
#     dnb.load_rasters()
#     dnb.build_regional_images()
#     dnb.add_padding()
#     dnb.save_regional_images()

In [2]:
# this script is used to clean gdp data

# Inflation data
# inflation = pd.read_excel("data/isc_reg.xls", skiprows=2, header=1)
# inflation = inflation.drop(columns=inflation.columns[0])
# inflation = inflation.rename(columns={inflation.columns[-1]: "region"})
# inflation = inflation[~inflation["region"].isin(["Ukraine", "oblasts"])]
# inflation = inflation.dropna()
# inflation["region"] = inflation["region"].map(region_map)
# inflation.columns = inflation.columns.astype(str)
# inflation = inflation.melt(id_vars="region", var_name="year", value_name="inflation")
# inflation.to_csv("data/inflation.csv", index=False)

# GDP data
gdp = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp = gdp.drop(columns=gdp.columns[0])
gdp = gdp.iloc[:, np.r_[18:36, -1]]
gdp = gdp.rename(columns={gdp.columns[-1]: "region"})
gdp = gdp[~gdp["region"].isin(["Ukrane", "oblasts"])]
gdp = gdp.dropna()
gdp["region"] = gdp["region"].map(region_map)
gdp["region"] = gdp["region"].fillna("Sevastopol")
gdp.columns = gdp.columns.astype(str)
gdp = gdp.rename(columns={gdp.columns[i]: gdp.columns[i][:4] for i in range(18)})
gdp = gdp.melt(id_vars="region", var_name="year", value_name="real_gdp_change")

# include only years from 2012 inclusive, exclude Sevastopol and the Autonomous Republic of Crimea
gdp = gdp[gdp["year"].astype(int) >= 2012]
gdp = gdp[~gdp["region"].isin(["Sevastopol", "Autonomous Republic of Crimea"])]

# set the value for the starting year to 100 (2012), NaN for the rest
gdp.loc[gdp["year"] == "2012", "real_gdp"] = 100
gdp = gdp.sort_values(by=["region", "year"])
gdp["real_gdp_change"] = gdp["real_gdp_change"] / 100

# reste the index
gdp = gdp.reset_index(drop=True)

# # calculate the real gdp
for i in range(1, gdp.shape[0]):

    # skip if the year is 2012
    if gdp.loc[gdp.index[i], "year"] == "2012":
        continue
    else:
        gdp.loc[gdp.index[i], "real_gdp"] = gdp.loc[gdp.index[i-1], "real_gdp"] * (gdp.loc[gdp.index[i], "real_gdp_change"])

# delete the real_gdp_change column
gdp = gdp.drop(columns="real_gdp_change")

# get the nominal gdp
gdp_nominal = pd.read_excel("data/ukr_reg_gdp.xls", skiprows=3, header=1)
gdp_nominal = gdp_nominal.iloc[:, np.r_[9, -1]]
gdp_nominal.columns = ["gdp_nominal", "region"]
gdp_nominal = gdp_nominal[~gdp_nominal["region"].isin(["Ukrane", "oblasts"])]
gdp_nominal = gdp_nominal.dropna()
gdp_nominal["region"] = gdp_nominal["region"].map(region_map)
gdp_nominal["region"] = gdp_nominal["region"].fillna("Sevastopol")

# merge nominal gdp to real gdp by region
gdp = gdp.merge(gdp_nominal, on="region")

# multiple the real gdp by the nominal gdp
gdp["real_gdp"] = gdp["real_gdp"] * gdp["gdp_nominal"]

# drop the nominal gdp column
gdp = gdp.drop(columns="gdp_nominal")

# for the region column, change all spaces to _
gdp["region"] = gdp["region"].str.replace(" ", "_")

# save the data
gdp.to_csv("data/clean_gdp.csv", index=False)


### Neural Network

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


# # Loading the MNIST dataset
# from keras.datasets import mnist
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# load clean gdp data
gdp = pd.read_csv("data/clean_gdp.csv")

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(gdp), 765, 1076, 4))
y = np.zeros(len(gdp))

# load the snow covered and snow free images, add them together and append to the list
for i in range(len(gdp)):

    # get year, region, and gdp
    year = gdp["year"][i]
    region = gdp["region"][i]
    gdp_value = gdp["real_gdp"][i]

    # get the file name
    file_name = f"{year}_{region}.h5"

    # load the image
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        nearnad_snow_cov = annual_region["NearNadir_Composite_Snow_Covered"][:]
        nearnad_snow_free = annual_region["NearNadir_Composite_Snow_Free"][:]
        offnad_snow_cov = annual_region["OffNadir_Composite_Snow_Covered"][:]
        offnad_snow_free = annual_region["OffNadir_Composite_Snow_Free"][:]

        # add the two images together
        # combined = snow_covered + snow_free

    # add the gdp value to y
    y[i] = gdp_value

    # append both images as two channels to to X
    X[i, :, :, 0] = nearnad_snow_cov
    X[i, :, :, 1] = nearnad_snow_free
    X[i, :, :, 2] = offnad_snow_cov
    X[i, :, :, 3] = offnad_snow_free

# print(X.shape)
# print(y.shape)

# Normalise the images
maximum = X.max()
X = X / maximum

# standardise gdp values
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std

# print(y.mean())
# print(maximum)

# change the format to a float16
X = X.astype(np.float16)
y = y.astype(np.float16)

In [4]:
# select 80% of the data for training, choose randomly
# X is the images, y is the gdp
train_size = int(0.8 * len(gdp))
test_size = len(gdp) - train_size

# select randomly train_size numbers from 0 to len(gdp)
train_indices = np.random.choice(len(gdp), train_size, replace=False)
test_indices = np.setdiff1d(np.arange(len(gdp)), train_indices)

# get the train data
X_train = X[train_indices]
y_train = y[train_indices]

# get the test data
X_test = X[test_indices]
y_test = y[test_indices]


In [5]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 4)))
# Start with Convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))  # Additional Conv layer
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))  # Additional Conv layer
model.add(MaxPooling2D((2, 2)))
# Flatten the results to feed into a dense layer
model.add(Flatten())
# Add dense layers (hidden layers)
model.add(Dense(128, activation='relu'))  # Upscaled dense layer
model.add(Dense(64, activation='relu'))   # Additional dense layer
# Output layer
model.add(Dense(1))
# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [12]:
model = Sequential()
# Resizing the images
model.add(Resizing(300, 440, input_shape=(765, 1076, 4)))

# Start with Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))  # Increased number of filters
model.add(MaxPooling2D((2, 2)))

# Flatten the results to feed into a dense layer
model.add(Flatten())

# Add dense layers (hidden layers)
model.add(Dense(256, activation='relu'))  # Increased the number of neurons
model.add(Dense(128, activation='relu'))  # Increased the number of neurons
model.add(Dense(64, activation='relu'))   # Kept as is for detailed feature extraction

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [6]:
# check the size of X_train and y_train
print(X_train.shape)
print(X_train[1, :, :, 0].max())

(200, 765, 1076, 4)
0.896


In [14]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)  # Assuming you have a validation split of 20%


MemoryError: Unable to allocate 1.96 GiB for an array with shape (160, 765, 1076, 4) and data type float32

In [8]:
# Step 6: Evaluate your model on the testing data
test_loss, test_mae = model.evaluate(X_test, y_test)
print('Test MAE:', test_mae) # mean absolute error
print('Test Loss:', test_loss)

2/2 [==============================] - 0s 110ms/step - loss: 0.0166 - mae: 0.0931
Test MAE: 0.09311747550964355
Test Loss: 0.016602778807282448


In [9]:
y_hat = model.predict(X_test).flatten()

print(y_test)
print(y_hat)

print(1 - y_hat/y_test)

print(np.mean(np.abs(1 - y_hat/y_test)))

2/2 [==============================] - 1s 120ms/step
[-0.3865  -0.3867  -0.3267   1.54     1.303    1.925    0.2595  -0.3655
 -0.4136  -0.3245   0.3606  -0.555   -0.5557  -0.5557  -0.445   -0.4287
 -0.514   -0.4731   0.3145   0.2273   0.2201   0.12396 -0.6284   0.267
  0.3418  -0.376   -0.381   -0.3293  -0.38     0.3176   0.0857   0.03027
 -0.06903 -0.523   -0.549   -0.5312  -0.4678  -0.585   -0.2986  -0.2491
 -0.542   -0.5083  -0.569   -0.542    0.05505  0.04834  0.02493  0.03052
  0.05487 -0.4001 ]
[-0.37542757 -0.3699261  -0.26612532  1.1491622   1.2016343   1.848887
  0.53231966 -0.5138074  -0.49566388 -0.37560466  0.47047287 -0.5861057
 -0.5505346  -0.58583593 -0.44584957 -0.48678458 -0.5754301  -0.49369138
  0.21880376  0.29775324  0.2946698  -0.30644888 -0.616561    0.08748098
  0.2635282  -0.4152276  -0.45284614 -0.4336341  -0.45927814  0.28671312
 -0.15302671 -0.23077467 -0.24244285 -0.5394714  -0.56930745 -0.56435776
 -0.5435089  -0.54009134 -0.354613   -0.325707   -0.5915505

In [9]:
# get the predictions from X_test
y_hat = model.predict(X_test).flatten()

# convert the predictions back to the original scale
# y_hat = y_hat * y_std + y_mean
# y_test = y_test * y_std + y_mean

# compute the mae
mae = np.mean(np.abs(y_test - y_hat))
print(mae)

# compute the mean percentage error
percentage_error = np.mean(100*np.abs((y_test - y_hat) / y_test))
print(percentage_error)


2/2 [==============================] - 0s 56ms/step
0.10076753842329474
36.946041487261475
